In [130]:
import keras
import tensorflow as tf
import numpy as np

In [4]:
model_path = 'BirdNET-Analyzer/checkpoints/V2.3/BirdNET_GLOBAL_3K_V2.3_Model_FP32.tflite'
tflite_interpreter = tf.lite.Interpreter(model_path)
tflite_interpreter.allocate_tensors()

In [10]:
tensor_details = tflite_interpreter.get_tensor_details()

In [18]:
[i for i, x in enumerate(tensor_details) if x['name'] == 'model/CONV_0/Conv2D']

[122]

In [22]:
tflite_conv0_weights = tflite_interpreter.tensor(122)()

In [24]:
model_path = 'BirdNET-Analyzer/checkpoints/V2.3/BirdNET_GLOBAL_3K_V2.3_Model'
model = keras.models.load_model(model_path, compile=False).model

In [37]:
keras_conv0_weights = model.get_layer('CONV_0').get_weights()[0]

In [38]:
tflite_conv0_weights.shape

(24, 4, 8, 1)

In [40]:
keras_conv0_weights.shape

(4, 8, 1, 24)

In [68]:
tflite_conv0_weights.transpose((1,2,3,0)) / keras_conv0_weights

array([[[[0.3054324 , 1.2421118 , 1.4848461 , 1.0409461 , 0.2827785 ,
          0.62431705, 0.49313167, 1.268903  , 0.54979515, 1.2540731 ,
          0.78386086, 0.9544975 , 0.6224919 , 1.3303999 , 0.6273121 ,
          0.2849848 , 0.69257116, 0.8139413 , 0.5641073 , 0.520027  ,
          1.7816279 , 0.94262123, 0.72143984, 1.193165  ]],

        [[0.3054324 , 1.2421118 , 1.4848462 , 1.0409461 , 0.2827785 ,
          0.62431705, 0.49313167, 1.268903  , 0.54979515, 1.2540731 ,
          0.78386086, 0.9544975 , 0.6224919 , 1.3303999 , 0.6273121 ,
          0.2849848 , 0.69257116, 0.8139413 , 0.5641073 , 0.520027  ,
          1.781628  , 0.9426211 , 0.72143984, 1.193165  ]],

        [[0.3054324 , 1.2421118 , 1.4848461 , 1.0409461 , 0.2827785 ,
          0.6243171 , 0.49313167, 1.268903  , 0.54979515, 1.2540731 ,
          0.78386086, 0.95449746, 0.6224919 , 1.3303999 , 0.6273121 ,
          0.2849848 , 0.69257116, 0.8139413 , 0.5641073 , 0.520027  ,
          1.781628  , 0.9426211 , 0.72

Interestingly, the weights are not exactly the same. In this particular layer (CONV_0) it seems like each channel of the output gets rescaled in the tflite model compared to the keras model. Maybe this is some optimization to reduce numerical error? That might be necessary for the lower precision versions like FP16.

In [92]:
for op in tflite_interpreter._get_ops_details():
    out = op['outputs'][0]
    for inp in op['inputs']:
        print('%d -> %d;' % (inp, out))

0 -> 158;
129 -> 158;
0 -> 159;
158 -> 159;
159 -> 160;
129 -> 160;
160 -> 161;
144 -> 161;
159 -> 162;
161 -> 162;
162 -> 163;
163 -> 164;
128 -> 164;
142 -> 164;
165 -> 167;
145 -> 167;
167 -> 168;
146 -> 168;
168 -> 169;
146 -> 169;
168 -> 170;
146 -> 170;
170 -> 171;
135 -> 171;
164 -> 172;
171 -> 172;
164 -> 173;
169 -> 173;
162 -> 174;
147 -> 174;
172 -> 174;
130 -> 174;
174 -> 175;
173 -> 175;
167 -> 176;
148 -> 176;
176 -> 177;
149 -> 177;
177 -> 178;
129 -> 178;
178 -> 179;
142 -> 179;
179 -> 180;
129 -> 180;
179 -> 181;
148 -> 181;
164 -> 182;
181 -> 182;
142 -> 183;
179 -> 183;
129 -> 183;
183 -> 184;
126 -> 184;
184 -> 185;
180 -> 185;
185 -> 186;
127 -> 186;
175 -> 187;
186 -> 187;
187 -> 188;
182 -> 188;
188 -> 189;
125 -> 189;
189 -> 190;
123 -> 190;
190 -> 191;
124 -> 191;
191 -> 192;
192 -> 193;
193 -> 194;
194 -> 195;
143 -> 195;
195 -> 196;
134 -> 196;
195 -> 197;
141 -> 197;
194 -> 198;
140 -> 198;
198 -> 199;
134 -> 199;
196 -> 200;
199 -> 200;
193 -> 201;
200 -> 2

In [125]:
alltensors = sorted([y for x in tensor_details for y in x['name'].split(';')])

In [122]:
ops = np.array([x['op_name'] for x in tflite_interpreter._get_ops_details()])
ops = np.unique(ops)

In [126]:
ops

array(['ADD', 'AVERAGE_POOL_2D', 'CAST', 'CONCATENATION', 'CONV_2D',
       'DEPTHWISE_CONV_2D', 'DIV', 'EXPAND_DIMS', 'FLOOR_DIV',
       'FULLY_CONNECTED', 'GATHER', 'LOGISTIC', 'MAXIMUM', 'MAX_POOL_2D',
       'MEAN', 'MUL', 'PACK', 'PAD', 'POW', 'RANGE', 'REDUCE_MAX',
       'REDUCE_MIN', 'REDUCE_PROD', 'RESHAPE', 'REVERSE_V2', 'RFFT2D',
       'SHAPE', 'SPLIT_V', 'SQUEEZE', 'STRIDED_SLICE', 'SUB', 'TRANSPOSE'],
      dtype='<U17')

In [128]:
for x in alltensors:
    print(x)





INPUT
Identity
model/ACT_0/Relu
model/ACT_POST/Relu
model/BLOCK_1-1_ACT_1/Sigmoid
model/BLOCK_1-1_ACT_1/mul
model/BLOCK_1-1_ACT_2/Sigmoid
model/BLOCK_1-1_ACT_2/mul
model/BLOCK_1-1_BN_1/FusedBatchNormV3
model/BLOCK_1-1_BN_1/FusedBatchNormV3
model/BLOCK_1-1_BN_2/FusedBatchNormV3
model/BLOCK_1-1_BN_2/FusedBatchNormV3
model/BLOCK_1-1_BN_2/FusedBatchNormV3
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D
model/BLOCK_1-1_CONV_1/Conv2D1
model/BLOCK_1-1_CONV_1/Conv2D1
model/BLOCK_1-1_CONV_2/depthwise
model/BLOCK_1-1_CONV_2/depthwise
model/BLOCK_1-1_CONV_2/depthwise1
model/BLOCK_1-1_CONV_3/Conv2D
